# Widgets II

In [226]:
import ipywidgets as widgets
from IPython.display import display,clear_output
import geopandas as gpd
import pandas as pd
import psycopg2
import folium

## Output Widget

In [201]:
out = widgets.Output()
out

Output()

Surprisingly we can't see anything. But we can start adding output to the output widget. Let us start with something simple such as our print statement

In [202]:
with out:
    print ('My name is Jay')
    print ('Python is cool')

What if we execute the code again 

In [203]:
with out:
    print ('My name is Jay')
    print ('Python is cool')

As you can see the new print statements get appended to the previous results. Let us try another example

In [204]:
out = widgets.Output()
out

Output()

In [205]:
with out:
    for idx in range(10):
        print (idx)

Now what if we want to overwrite the content rather than appending. We can use the clear_output function available in the Ipython display module

In [206]:
out = widgets.Output()
out

Output()

In [207]:
with out:
    for idx in range(3500):
        clear_output()
        print (idx)

As you can see there is considerable flickering. Inorder to avoid that we can set wait parameter to True in clear_output() function

In [208]:
out = widgets.Output()
out

Output()

In [209]:
with out:
    for idx in range(3500):
        clear_output(wait=True)
        print (idx)

Let us replicate our small geocoding tool with a small change. In this case we will display the coordinates at the end

In [210]:
def geocode(b):
    results = gpd.tools.geocode([textBox.value])
    with out:
        clear_output(wait=True)
        print (results)

textBox = widgets.Text(
    value='',
    placeholder='Type your address here',
    description='Address:',
    disabled=False   
)
button = widgets.Button(
    description='Click',
    disabled=False,
    button_style='Primary'
)
out = widgets.Output()
display(textBox)
button.on_click(geocode)
display(button)
display(out)

Text(value='', description='Address:', placeholder='Type your address here')

Button(button_style='primary', description='Click', style=ButtonStyle())

Output()

Eventhough we can see the entire output, it is messy. Let us just print the latitude and longitude

In [211]:
def geocode(b):
    results = gpd.tools.geocode([textBox.value])
    with out:
        clear_output(wait=True)
        print (results.geometry.x[0],results.geometry.y[0])

textBox = widgets.Text(
    value='',
    placeholder='Type your address here',
    description='Address:',
    disabled=False   
)
button = widgets.Button(
    description='Click',
    disabled=False,
    button_style='Primary'
)
out = widgets.Output()
display(textBox)
button.on_click(geocode)
display(button)
display(out)

Text(value='', description='Address:', placeholder='Type your address here')

Button(button_style='primary', description='Click', style=ButtonStyle())

Output()

### Pandas DataFrame in Output

You can display pandas dataframe in an output display. We will first show a simple example and then an example involving the database

In [212]:
data = pd.DataFrame([['Jay',35,'M'],['Sam',24,'M'],['Anne',22,'F']],columns=['Name','Age','Sex'])

In [213]:
out = widgets.Output()
with out:
    display(data)
out

Output()

Now let us create a small application which displays the total earthquakes for various years for a selected state. So we will use the drop down widget for the states.

In [214]:
conn = psycopg2.connect('dbname=nyc user=geospatial password=geospatial2023 host=ghhlibrary')

In [ ]:
states = pd.read_sql_query('select name from us_states order by name',conn)

Now let us create our UI

In [ ]:
def getEarthQuakeData(change):
    state = change['new']
    data = pd.read_sql_query(f"select e.year::text,count(e.gid) as total from earth_quakes e,us_states u where u.name='{state}' and st_contains(u.geom,e.geom) group by e.year order by e.year",conn)
    print (data)
    with out:
        clear_output(wait=True)
        display(data)
statesSelect = widgets.Dropdown(
    options=states.name.values,
    value=None,
    description='States:',
    disabled=False,
)
out = widgets.Output()
statesSelect.observe(getEarthQuakeData,names='value')
display(statesSelect)
display(out)

### Plotting in Output

In this example we are going to show a barchart of number of homicides in the different boroughs in nyc for different years. 

Let us query the years for which we would want the data

In [159]:
years = pd.read_sql_query('select distinct year::text as year from nyc_homicides order by year',conn)

C:\Users\jxa421\AppData\Local\Temp\ipykernel_83304\2164346758.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  years = pd.read_sql_query('select distinct year::text as year from nyc_homicides order by year',conn)


Now let us create our UI which consists of a slider and an output widget (where we will have the chart)

In [44]:
def homicidesBorough(change):
    year = change['new']
    data = pd.read_sql_query(f"select boroname,count(gid) as total_homicides from nyc_homicides where year={year} group by boroname order by boroname",conn)
    with out:
        clear_output(wait=True)
        display(data)
out = widgets.Output()
yearSlider = widgets.SelectionSlider(
    options=years.year.values,
    value=years.year.values[0],
    description='Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
yearSlider.observe(homicidesBorough,names='value')
display(out)
display(yearSlider)

Output()

SelectionSlider(continuous_update=False, description='Year:', options=('2003', '2004', '2005', '2006', '2007',…

So that's the dataframe, let us create a chart

In [50]:
def homicidesBorough(change):
    year = change['new']
    data = pd.read_sql_query(f"select boroname,count(gid) as total_homicides from nyc_homicides where year={year} group by boroname order by boroname",conn)
    with out:
        clear_output(wait=True)
        ax = data.plot.bar(x='boroname',ylabel='Total Homicides',legend=False)
        display(ax.get_figure())
out = widgets.Output()
yearSlider = widgets.SelectionSlider(
    options=years.year.values,
    value=years.year.values[0],
    description='Year:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True
)
yearSlider.observe(homicidesBorough,names='value')
display(yearSlider)
display(out)

SelectionSlider(description='Year:', options=('2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', …

Output()

Now let us look into another example. In this example we are going to create a longitudnal plot for the various counties in Ohio (selection based on counties). So let us start with the basics. We will create a drop down of counties in Ohio and an output widget for the graph

In [51]:
countiesOhio = pd.read_sql_query("select name,geoid from us_counties where statefp='39' order by name",conn)

C:\Users\jxa421\AppData\Local\Temp\ipykernel_83304\598678902.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  countiesOhio = pd.read_sql_query("select name,geoid from us_counties where statefp='39' order by name",conn)


Now let us also read in the different year-month for our covid dataset. We will use this as our x-axis. 

In [ ]:
yearMonth = pd.read_sql_query("select distinct case_month from covid_oh_county_totals order by case_month",conn)

Now let us create UI and then create a function for capturing the drop down change event

In [69]:
def countySelected(change):
    county = change['new']
    countyName = countiesOhio[countiesOhio.geoid==county].name.values[0]
    data = pd.read_sql_query(f"select case_month,total from covid_oh_county_totals where county_fips_code = '{county}' order by case_month",conn)
    data = yearMonth.merge(data,on='case_month',how='left').fillna(0)
    with out:
        clear_output(wait=True)
        ax = data.plot(figsize=(10,8),x='case_month',xlabel='Year Month',ylabel='Total Cases',title=f'Covid-19 case distribution for {countyName}',legend=False)
        display(ax.get_figure())
    
out = widgets.Output()
countiesSelect = widgets.Dropdown(
    options=countiesOhio.values.tolist(),
    value=None,
    description='OH Counties:',
    disabled=False,
)
countiesSelect.observe(countySelected,names='value')
display(out)
display(countiesSelect)

Output()

Dropdown(description='OH Counties:', options=(['Adams', '39001'], ['Allen', '39003'], ['Ashland', '39005'], ['…

A small thing you can notice is that the y-axis shifts often. If you want to avoid it, you need to use the ylim property of the plot function. But before that you need to find the maximum value for the entire dataset. 

In [84]:
casemax = pd.read_sql_query("select max(total) as maxtotal from covid_oh_county_totals",conn)
highest = casemax.maxtotal.values[0]

C:\Users\jxa421\AppData\Local\Temp\ipykernel_83304\1969231126.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  casemax = pd.read_sql_query("select max(total) as maxtotal from covid_oh_county_totals",conn)


In [88]:
def countySelected(change):
    county = change['new']
    countyName = countiesOhio[countiesOhio.geoid==county].name.values[0]
    data = pd.read_sql_query(f"select case_month,total from covid_oh_county_totals where county_fips_code = '{county}' order by case_month",conn)
    data = yearMonth.merge(data,on='case_month',how='left').fillna(0)
    with out:
        clear_output(wait=True)
        ax = data.plot(ylim=(0,highest),figsize=(10,8),x='case_month',xlabel='Year Month',ylabel='Total Cases',title=f'Covid-19 case distribution for {countyName}',legend=False)
        display(ax.get_figure())
    
out = widgets.Output()
countiesSelect = widgets.Dropdown(
    options=countiesOhio.values.tolist(),
    value=None,
    description='OH Counties:',
    disabled=False,
)
countiesSelect.observe(countySelected,names='value')
display(out)
display(countiesSelect)

Output()

Dropdown(description='OH Counties:', options=(['Adams', '39001'], ['Allen', '39003'], ['Ashland', '39005'], ['…

But we are suddenly faced with the challenge of counties having very small and very high values. In-order to by pass that we can use log scale for y axis

In [114]:
def countySelected(change):
    county = change['new']
    countyName = countiesOhio[countiesOhio.geoid==county].name.values[0]
    data = pd.read_sql_query(f"select case_month,total from covid_oh_county_totals where county_fips_code = '{county}' order by case_month",conn)
    data = yearMonth.merge(data,on='case_month',how='left').fillna(0)
    with out:
        clear_output(wait=True)
        ax = data.plot(logy=True,ylim=((10**0,10**5)),figsize=(10,8),x='case_month',xlabel='Year Month',ylabel='Total Cases',title=f'Covid-19 case distribution for {countyName}',legend=False)
        display(ax.get_figure())
    
out = widgets.Output()
countiesSelect = widgets.Dropdown(
    options=countiesOhio.values.tolist(),
    value=None,
    description='OH Counties:',
    disabled=False,
)
countiesSelect.observe(countySelected,names='value')
display(out)
display(countiesSelect)

Output()

Dropdown(description='OH Counties:', options=(['Adams', '39001'], ['Allen', '39003'], ['Ashland', '39005'], ['…

If you want to mathematically calculate the upper limit when using the log10 scale then you can use the following code,
```python
import math
upper = math.ceil(math.log10(highest))
```

Now let us do one final longitudnal plot using our covid data. In this example we are going to porvide a multi select drop down where the user can select multiple counties (only up to 4 as we would want to restrict the number of counties selected for aesthetic purposes).

In [162]:
def countySelect(change):
    if len(change['new'])>0 and len(change['new'])<5:
        errorLabel.value = ''
        if len(change['new'])==1:
            query = f"select c.case_month,u.name,c.total from covid_oh_county_totals c,us_counties u where c.county_fips_code = '{change['new'][0]}' and u.geoid=c.county_fips_code order by case_month"
        else:
            query = f"select c.case_month,u.name,c.total from covid_oh_county_totals c,us_counties u where county_fips_code in {change['new']} and u.geoid=c.county_fips_code order by case_month"
        data = pd.read_sql_query(query,conn)
        data = data.pivot(index='case_month',columns='name',values='total').fillna(0).reset_index()
        data = yearMonth.merge(data,on='case_month',how='left').fillna(0)
        with out:
            clear_output(wait=True)
            ax = data.plot(ylim = (10**0,10**5),logy=True,figsize=(10,8),x='case_month',xlabel='Case Month',ylabel='Total Cases (log10)',title='Covid-19 case distribution')
            display(ax.get_figure())
    else:
        errorLabel.value='Select atleast 1 county and atmost 4 counties'
        
out = widgets.Output()
countiesMultipleSelect = widgets.SelectMultiple(
    options=countiesOhio.values.tolist(),
    value=[],
    rows=10,
    description='OH Counties:',
    disabled=False
)
countiesMultipleSelect.observe(countySelect,names='value')
errorLabel = widgets.Label(value='')
display(out)
display(countiesMultipleSelect)
display(errorLabel)

Output()

SelectMultiple(description='OH Counties:', options=(['Adams', '39001'], ['Allen', '39003'], ['Ashland', '39005…

Label(value='')

Until now we have only used display function to show our widgets and our widgets are just stacked on top of each others. Now we will look into using layout objects for arranging our widgets.

## HBox and VBox

### HBox

HBox or horizontal box allows you to arrange your widgets horizontally. Let us stack two buttons horizontally

In [118]:
button1 = widgets.Button(
    description='Click Me',
    disabled=False,
    button_style='Primary'
)
button2 = widgets.Button(
    description='No Click Me',
    disabled=False,
    button_style='Primary'
)
display(button1)
display(button2)

Button(button_style='primary', description='Click Me', style=ButtonStyle())

Button(button_style='primary', description='No Click Me', style=ButtonStyle())

As yuo can see if you are just using display both the buttons will be stacked on top of each other. Now let us use an HBox

In [120]:
button1 = widgets.Button(
    description='Click Me',
    disabled=False,
    button_style='Primary'
)
button2 = widgets.Button(
    description='No Click Me',
    disabled=False,
    button_style='Primary'
)
myBox = widgets.HBox([button1,button2])
display(myBox)

As you can see, now you have the buttons arranged horizontally. 

As an example let us re-create our covid cases example with the widgets stacked horizontally

In [122]:
def countySelected(change):
    county = change['new']
    countyName = countiesOhio[countiesOhio.geoid==county].name.values[0]
    data = pd.read_sql_query(f"select case_month,total from covid_oh_county_totals where county_fips_code = '{county}' order by case_month",conn)
    data = yearMonth.merge(data,on='case_month',how='left').fillna(0)
    with out:
        clear_output(wait=True)
        ax = data.plot(logy=True,ylim=((10**0,10**5)),figsize=(10,8),x='case_month',xlabel='Year Month',ylabel='Total Cases',title=f'Covid-19 case distribution for {countyName}',legend=False)
        display(ax.get_figure())
    
out = widgets.Output()
countiesSelect = widgets.Dropdown(
    options=countiesOhio.values.tolist(),
    value=None,
    description='OH Counties:',
    disabled=False,
)
countiesSelect.observe(countySelected,names='value')
myBox = widgets.HBox([countiesSelect,out])
display(myBox)

### VBox

VBox or vertical box allows you to arrange your widgets vertically. Let us stack two buttons vertically

In [123]:
button1 = widgets.Button(
    description='Click Me',
    disabled=False,
    button_style='Primary'
)
button2 = widgets.Button(
    description='No Click Me',
    disabled=False,
    button_style='Primary'
)
myBox = widgets.VBox([button1,button2])
display(myBox)

Let us use the nyc_homicide example that we have seen before

In [160]:
def homicidesBorough(change):
    year = change['new']
    data = pd.read_sql_query(f"select boroname,count(gid) as total_homicides from nyc_homicides where year={year} group by boroname order by boroname",conn)
    with out:
        clear_output(wait=True)
        ax = data.plot.bar(x='boroname',ylabel='Total Homicides',legend=False)
        display(ax.get_figure())
out = widgets.Output()
yearSlider = widgets.SelectionSlider(
    options=years.year.values,
    value=years.year.values[0],
    description='Year:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True
)
yearSlider.observe(homicidesBorough,names='value')
myBox = widgets.VBox([out,yearSlider])
display(myBox)

### Combining VBox and HBox

The real power of layouts is when you can combine the different objects. For example you can have a VBox inside an HBox as well as viceversa. 

In this example we are going to show the rail incidents for various counties in different years as a temporal chart. It is always better to have a mental visualization of the page you want to create first

![sample](images/sample.png)

We will create the two drop downs and then add them to a vertical box. Then we will create an output widget and then add the vertical box and the output widget to a horizonal box. Then we will write the necessary event capture statements. 

In [164]:
states = pd.read_sql_query(r'select name,state_fips from us_states order by name',conn)
#we will create year ranges using the minimum and maximum date for the rail incidents
maxminIncident = pd.read_sql_query(r'select max(EXTRACT(YEAR from date)) as max_year,min(EXTRACT(YEAR from date)) as min_year from rail_incidents',conn)
#generate years series
yearseries = pd.Series(range(int(maxminIncident.min_year.values[0]),int(maxminIncident.max_year.values[0])))
#create a dataframe with single column year containing the different years
yearsFrame = pd.DataFrame(yearseries,columns=['year'])
def stateSelected(change):
    selectedState = change['new']
    #now let us query all the counties for the state. Unfortunately no state name in counties table, we need a join
    counties = pd.read_sql_query(f"select c.name,c.geoid from us_counties c where c.statefp='{selectedState}' order by c.name",conn)
    countiesSelect.options = counties.values.tolist()
    
def countySelected(change):
    selectedCounty = change['new']
    incidents = pd.read_sql_query(f"select EXTRACT(YEAR from date) as year,count(id) as total_incidents from rail_incidents where fips_county='{selectedCounty}' group by year order by year",conn)
    #now merge with the dateframe and fill null values as zero
    data = yearsFrame.merge(incidents,on='year',how='left').fillna(0)
    with out:
        clear_output(wait=True)
        ax = data.plot(figsize=(10,8),x='year',xlabel='Year',ylabel='Total Incidents',title=f'Yearly Incident Distribution',legend=False)
        display(ax.get_figure())
    
statesSelect = widgets.Dropdown(
    options=states.values.tolist(),
    value=None,
    description='States:',
    disabled=False,
)
countiesSelect = widgets.Dropdown(
    options=[],
    value=None,
    description='Counties:',
    disabled=False,
)
statesSelect.observe(stateSelected,names='value')
countiesSelect.observe(countySelected,names='value')
out = widgets.Output()
dropDowns = widgets.VBox([statesSelect,countiesSelect])
allWidgets = widgets.HBox([dropDowns,out])
display(allWidgets)

C:\Users\jxa421\AppData\Local\Temp\ipykernel_83304\1801771992.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  states = pd.read_sql_query(r'select name,state_fips from us_states order by name',conn)
C:\Users\jxa421\AppData\Local\Temp\ipykernel_83304\1801771992.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  maxminIncident = pd.read_sql_query(r'select max(EXTRACT(YEAR from date)) as max_year,min(EXTRACT(YEAR from date)) as min_year from rail_incidents',conn)


### Plotting Maps With Output

Similar to bar and line plots we can also create maps (using geopandas) and render them in output.
As a simple example we are going to use our old geocoder to create a live geocoding tool. Rather than printing the location we are going to use the explore method th

In [183]:
def geocode(b):
    results = gpd.tools.geocode([textBox.value])
    with out:
        clear_output(wait=True)
        display(results.explore(marker_kwds={'radius':5}))

textBox = widgets.Text(
    value='',
    placeholder='Type your address here',
    description='Address:',
    disabled=False   
)
button = widgets.Button(
    description='Click',
    disabled=False,
    button_style='Primary'
)
out = widgets.Output()
display(out)
display(textBox)
button.on_click(geocode)
display(button)

Output()

Text(value='', description='Address:', placeholder='Type your address here')

Button(button_style='primary', description='Click', style=ButtonStyle())

Now let us create choropleth maps for the census tracts in various counties in US. We will use HBox and VBox

In [190]:
allstates = pd.read_sql_query(r'select name,state_fips from us_states order by name',conn)
def stateSelected(change):
    selectedState = change['new']
    #now let us query all the counties for the state. Unfortunately no state name in counties table, we need a join
    counties = pd.read_sql_query(f"select c.name,c.geoid from us_counties c where c.statefp='{selectedState}' order by c.name",conn)
    countiesSelect.options = counties.values.tolist()
    
def countySelected(change):
    selectedCounty = change['new']
    sviTracts = gpd.read_postgis(f"select fips,geom,rpl_themes from us_tract_svi where stcnty ='{selectedCounty}' and rpl_themes>=0",conn)
    with out:
        clear_output(wait=True)
        display(sviTracts.explore(column='rpl_themes', cmap='RdYlGn_r'))
    
statesSelect = widgets.Dropdown(
    options=allstates.values.tolist(),
    value=None,
    description='States:',
    disabled=False,
)
countiesSelect = widgets.Dropdown(
    options=[],
    value=None,
    description='Counties:',
    disabled=False,
)
statesSelect.observe(stateSelected,names='value')
countiesSelect.observe(countySelected,names='value')
out = widgets.Output(layout=widgets.Layout(width='75%'))
dropDowns = widgets.VBox([statesSelect,countiesSelect],layout=widgets.Layout(width='25%'))
allWidgets = widgets.HBox([dropDowns,out])
display(allWidgets)

C:\Users\jxa421\AppData\Local\Temp\ipykernel_83304\3171948885.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  allstates = pd.read_sql_query(r'select name,state_fips from us_states order by name',conn)


Finally we will create a map of the different neighborhoods in NYC and then display the distribution of shooting or homicide based on user selection. 

In [198]:
def changeLayer(change):
    layer = change['new']
    if layer=='Shooting':
        color = 'Reds'
        query = f'select t.name,t.boroname,t.geom,count(s.gid) as total from nyc_neighborhoods t left join nypd_shooting s on st_contains(t.geom,s.geom) group by t.name,t.boroname,t.geom'
    else:
        color = 'Blues'
        query = f'select t.name,t.boroname,t.geom,count(s.gid) as total from nyc_neighborhoods t left join nyc_homicides s on st_contains(t.geom,s.geom) group by t.name,t.boroname,t.geom'
    data = gpd.read_postgis(query,conn)
    with out:
        clear_output(wait=True)
        display(data.explore(column='total', cmap=color))
    
selection = widgets.RadioButtons(
    options=['Shooting', 'Homicide'],
    description='Layer:',
    disabled=False,
    value=None,
    layout=widgets.Layout(width='25%')
)
selection.observe(changeLayer,names='value')
out = widgets.Output(layout=widgets.Layout(width='75%'))
allWidgets = widgets.HBox([selection,out])
display(allWidgets)

Now let us create a map where we switch on an off multiple layers. Here we are going to show schools, shooting locations, homicides, and subway stations for Harlem neighborhood in Manhattan. 

First let us create a list of layer names and then loop through it to create the checkboxes and save them to a list. Then we will create a draw method which will use the status of the check boxes to add the layers

In [254]:
def draw(change):
    newmap = folium.Map(location=[40.712778, -74.006111],tiles='CartoDB dark_matter')
    for i in range(len(checkboxes)):
        geodata = None
        #if shooting is true
        if checkboxes[i].description=='Shooting' and checkboxes[i].value:
            geodata = gpd.read_postgis(f"select s.gid,s.geom from nypd_shooting s,nyc_neighborhoods nb where nb.boroname='Manhattan' and nb.name='Harlem'and st_contains(nb.geom,s.geom)",conn)
            style_kwds = {'fillColor':'red','stroke':False}
        elif checkboxes[i].description=='Homicide' and checkboxes[i].value:
            geodata = gpd.read_postgis(f"select s.gid,s.geom from nyc_homicides s,nyc_neighborhoods nb where nb.boroname='Manhattan' and nb.name='Harlem'and st_contains(nb.geom,s.geom)",conn)
            style_kwds = {'fillColor':'orange','stroke':False}
        elif checkboxes[i].description=='School' and checkboxes[i].value:
            geodata = gpd.read_postgis(f"select s.gid,s.geom from nyc_schools s,nyc_neighborhoods nb where nb.boroname='Manhattan' and nb.name='Harlem'and st_contains(nb.geom,s.geom)",conn)
            style_kwds = {'fillColor':'green','stroke':False}
        elif checkboxes[i].description=='Subway' and checkboxes[i].value:
            geodata = gpd.read_postgis(f"select s.gid,s.geom from nyc_subway_stations s,nyc_neighborhoods nb where nb.boroname='Manhattan' and nb.name='Harlem'and st_contains(nb.geom,s.geom)",conn)
            style_kwds = {'fillColor':'blue','stroke':False}
        if geodata is not None:
            geodata.explore(marker_kwds={'radius':3},style_kwds=style_kwds,m=newmap)
    with out:
        clear_output(wait=True)
        display(newmap)

layers = ['Shooting','Homicide','School','Subway']
checkboxes = []
for layer in layers:
    checkBox = widgets.Checkbox(
    value=False,
    description=layer,
    disabled=False,
    indent=False
    )
    checkBox.observe(draw,names='value')
    checkboxes.append(checkBox)
cbox = widgets.VBox(checkboxes,layout=widgets.Layout(width='25%'))
out = widgets.Output(layout=widgets.Layout(width='75%'))
hb = widgets.HBox([cbox,out])
display(hb)

In the next lecture we will look into more mapping examples as well as ways to interact with the map.

In [199]:
conn.close()